In [1]:
import folium
import os

In [7]:
data = os.path.join('Subway Stations.geojson')

In [4]:
#Create a map object, focused on NYC:
m = folium.Map(location=[40.7142700, -74.0059700], zoom_start=13)

In [8]:
folium.GeoJson(data, name='Train Stations').add_to(m)

In [14]:
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d

/home/matheus/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
#File with station locations in NYC:
libs = pd.read_csv('test.csv')

#Create lists to hold coordinates and popups:
coords = []
popups = []
icons = []

In [17]:
#For each row in the CSV, pull out the latitude, longitude, and name of the library:
for index, row in libs.iterrows():
    words = row['the_geom'].split(" ")
    lat = float(words[2][:-1])
    lon = float(words[1][1:])
    name = row['NAME']
    #Add the [lat,lon] to list of coordinates:
    coords.append([lat,lon])
#Use scipy to make the voronoi diagram:
vor = Voronoi(coords, qhull_options='Qbb Qc Qx')

In [19]:
#Use geojson file to write out the features
from geojson import FeatureCollection, Feature, Polygon

In [26]:
#The output file, to contain the Voronoi diagram we computed:
vorJSON = open('Subway_Stations.geojson', 'w')
point_voronoi_list = []
feature_list = []

In [27]:
for region in range(len(vor.regions)-1):
    vertex_list = []
    for x in vor.regions[region]:
        if x == -1:
            break;
        else:
            #Get the vertex out of the list, and flip the order for folium:
            vertex = vor.vertices[x]
            vertex = (vertex[1], vertex[0])
        vertex_list.append(vertex)
    #Save the vertex list as a polygon and then add to the feature_list:
    polygon = Polygon([vertex_list])
    feature = Feature(geometry=polygon, properties={})
    feature_list.append(feature)

In [30]:
#Write the features to the new file:
feature_collection = FeatureCollection(feature_list)
print (feature_collection, file=vorJSON)
vorJSON.close()

ValueError: I/O operation on closed file.

In [23]:
#Add Voronoid diagram to map object
folium.GeoJson(data, name='Train Stations-Vor').add_to(m)

In [24]:
display(m)